In [1]:
import json
from tronpy import Tron
from tronpy.providers.http import HttpProvider
from tronpy.keys import PrivateKey
import os
from tronpy.keys import PublicKey

from tronpy.contract import Contract


In [3]:
NETWORK = "development"  # or "mainnet"
NETWORKS = {
    "nile": "https://nile.trongrid.io",
    "mainnet": "https://api.tronstack.io",
    "shasta": "https://api.shasta.trongrid.io",
    "development": "http://localhost:9090",
}

# Your private key (keep this secret and secure!)
# PRIVATE_KEY = "your_private_key_here"
# read from ../.env file

PRIVATE_KEY = "6d28b0701dbad9945ec26518d71f04f7361c6a5a23a19025a23b9bc68a7afeb3"

# Path to your compiled contract JSON files
CONTRACT_PATH = "../build/contracts/"

# Path to save deployed addresses
DEPLOY_ADDRESS_PATH = "./build/contractAddresses/"

# client = Tron(network="development")



In [ ]:

client = Tron()
account = PrivateKey(bytes.fromhex(PRIVATE_KEY))

In [72]:
def load_contract(contract_name):
    with open(f"{CONTRACT_PATH}{contract_name}.json") as file:
        contract_json = json.load(file)
    abi = contract_json['abi']
    bytecode = contract_json['bytecode']
    
    # Ensure bytecode is in correct format (remove '0x' prefix if present)
    if bytecode.startswith('0x'):
        bytecode = bytecode[2:]
    
    # Verify that bytecode is valid hexadecimal
    try:
        bytes.fromhex(bytecode)
    except ValueError:
        raise ValueError(f"Invalid bytecode format in {contract_name}.json")
    
    return abi, bytecode


from tronpy import Tron
from tronpy.keys import PrivateKey
from tronpy.contract import Contract

def deploy_contract(contract_name, *args):
    abi, bytecode = load_contract(contract_name)
    
    # Derive the address from the public key
    sender_address = account.public_key.to_base58check_address()
    
    # Confirm deployment
    confirm = input(f"Deploy {contract_name}? (y/n): ")
    if confirm.lower() != 'y':
        print(f"Skipping deployment of {contract_name}")
        return None

    # Create the contract object
    contract = Contract(
        client=client,
        bytecode=bytecode,
        abi=abi,
        owner_address=sender_address,
        origin_address=sender_address
    )

    # Encode constructor parameters
    constructor_params = contract.constructor.encode_parameter(*args)

    # Create the transaction
    tx = contract.deploy().fee_limit(100_000_000).build()

    # Sign the transaction using the PrivateKey object
    signed_tx = account.sign(tx)
    result = client.broadcast(signed_tx)
    
    # Wait for the transaction to be confirmed
    result = client.get_transaction_info(result['txid'])
    
    contract_address = result['contract_address']
    print(f"{contract_name} deployed at: {contract_address}")
    return contract_address

def save_addresses(addresses):
    if not os.path.exists(DEPLOY_ADDRESS_PATH):
        os.makedirs(DEPLOY_ADDRESS_PATH)
    with open(f"{DEPLOY_ADDRESS_PATH}{NETWORK}_addresses.json", 'w') as file:
        json.dump(addresses, file, indent=2)
    print(f"Addresses saved to {DEPLOY_ADDRESS_PATH}{NETWORK}_addresses.json")


In [73]:
deployed_addresses = {}



In [ ]:

# Deploy MasterAccessControl

master_access_control = deploy_contract("MasterAccessControl")
if master_access_control:
    deployed_addresses['MasterAccessControl'] = master_access_control

In [ ]:
deployed_addresses

In [ ]:
def deploy_contract(contract_name, *args):
    abi, bytecode = load_contract(contract_name)
    
    # Remove the estimation part
    # estimated_energy = client.estimate_energy(account.address, bytecode)
    # estimated_bandwidth = len(bytecode) + 64  # Approximate estimation
    
    # print(f"Estimated energy for {contract_name}: {estimated_energy}")
    # print(f"Estimated bandwidth for {contract_name}: {estimated_bandwidth}")
    
    # Confirm deployment
    confirm = input(f"Deploy {contract_name}? (y/n): ")
    if confirm.lower() != 'y':
        print(f"Skipping deployment of {contract_name}")
        return None

    # Proceed with deployment as before
    tx = client.trx.deploy_contract(
        account.address,
        abi,
        bytecode,
        *args,
        fee_limit=100_000_000
    )
    
    # Sign and broadcast the transaction
    signed_tx = account.sign_transaction(tx)
    result = client.trx.broadcast(signed_tx)
    
    # Wait for the transaction to be confirmed
    result = client.trx.wait_for_transaction(result['txid'])
    
    contract_address = result['contract_address']
    print(f"{contract_name} deployed at: {contract_address}")
    return contract_address

In [ ]:
from tronpy import Tron

client = Tron(network='private', endpoint_uri='http://127.0.0.1:8090')

block = client.get_current_block()
print('Current Block:', block)
